# NPI-4: NPI is numeric

Description: check if all NPIs are numeric. Test will pass if it is true.

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, test will print out the number of distinct npi that are not numeric.


def npi_4(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')   
        
        # MySQL query to get failed NPIs
        query = ('''
                SELECT t1.c1 - t2.c2
                FROM (
                  (SELECT count(*) AS c1
                  FROM {db}.{t1}) AS t1,
                  (SELECT count(*) AS c2
                  FROM {db}.{t1}
                  WHERE {col1} REGEXP '^[0-9]+$' OR {col1} = 'GLOBAL') AS t2
                );
        '''.format(db = db_name, t1 = table_name, col1 = col_name))
        
        cur.execute(query)

        print ("The number of distinct {} that are not numeric is:".format(col_name) + '\n')
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            print ('\n')
        if row[i] == 0:
            print ('Npi-4 test result: PASS')
        else:
            print ('Npi-4 test result: FAIL')
            
        cur.close()
        connection.close()

## Test Example

SELECT t1.c1 - t2.c2a
FROM (
  (SELECT count(*) AS c1
  FROM drg_npi.drg_npi_16_17XX) AS t1,
  (SELECT count(*) AS c2
  FROM drg_npi.drg_npi_16_17XX
  WHERE npi REGEXP '^[0-9]+$') AS t2
); # total nrow - isnumeric nrow = 0

In [3]:
npi_4('_amy', 'test_data_good', 'npi')

Test file: _amy.test_data_good


The number of distinct npi that are not numeric is:

0


Npi-4 test result: PASS


In [4]:
npi_4('_amy', 'test_data_bad1', 'npi')

Test file: _amy.test_data_bad1


The number of distinct npi that are not numeric is:

943


Npi-4 test result: FAIL


In [3]:
npi_4('client_celgene',' HCP_Geographic_distribution_of_MM_Patients', '`Physician NPI Number`')

Test file: client_celgene. HCP_Geographic_distribution_of_MM_Patients


The number of distinct `Physician NPI Number` that are not numeric is:

0


Npi-4 test result: PASS
